In [104]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import twitter
from twitter import error
import json
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [105]:
#client = datastore.Client()
#client = language.LanguageServiceClient()

In [106]:
api = twitter.Api(consumer_key="NOrVFOrKaGpOsD1LHjlDmD9lA",
                  consumer_secret="8Lg3hdsmIbEn9Q1fLTiWbCqHD7vp2OSc21mvTGElnudCr5jM7p",
                  access_token_key="49683260-3LQ6XoNAFi9R7MVaSGU1igwNxJkfxL6aFpUJfe7pG",
                  access_token_secret="zNXNyrewzVYC2kIqQ1GlGQoLIDHmTJtjXpMFZhwKGxYb8")

In [107]:
#https://dev.twitter.com/rest/reference/get/search/tweets
# https://python-twitter.readthedocs.io/en/latest/searching.html
# https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively
sinceId = None
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
max_id = -1
tweetCount = 0
results = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")
#for a in results:
#    if(a.coordinates is not None):
#print(a.coordinates)
    
forma = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.coordinates,
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) if a.coordinates is None else (a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          str(a.coordinates['coordinates']),
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) for a in results]
print(forma)
#https://dev.twitter.com/overview/api/tweets#obj-coordinates

[('Mon Sep 18 07:30:05 +0000 2017', "Home button or no home button 😉 it's still dangerous to use while driving. 📱 #AppleEvent #iPhone #Apple #iPhone8… https://t.co/6PLGEsKzws", 31, 21, 909680911515602945, 'THINK! road safety', 'THINKgovuk', 'Official THINK! road safety channel. We encourage safer behaviour to reduce the number of deaths/injuries on our roads. Twitter policy http://t.co/oDLlotPz8f', 1322025240, 'Great Britain', 513, 22277, 448, 2294, 'Tue Apr 02 08:31:05 +0000 2013', True, 'Edinburgh', None, None, None, None), ('Thu Sep 21 15:25:56 +0000 2017', "iOS 11 is here... and it's 🔥 #ios11 #iphone #iphoneX #apple https://t.co/iQlPkKrDy7", 24, 21, 910887828636266496, 'HubSpot', 'HubSpot', "The world's leading inbound marketing & sales platform with 31K customers in 90+ countries. For technical queries contact @HubSpotSupport.", 14458280, 'Cambridge, MA', 26325, 780793, 46113, 75905, 'Mon Apr 21 04:17:34 +0000 2008', True, 'Eastern Time (US & Canada)', None, None, None, None), ('S

In [108]:
"""
while tweetCount < 300:
    try:
        if(max_id <= 0):
            if (not sinceId):
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")
            else:
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en", 
                                  since_id=sinceId)
        else:
            if(not sinceId):
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",max_id=str(max_id - 1))
            else:
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",max_id=str(max_id - 1), since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
            print(tweet)
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except api.error.TwitterError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break
"""

'\nwhile tweetCount < 300:\n    try:\n        if(max_id <= 0):\n            if (not sinceId):\n                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")\n            else:\n                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en", \n                                  since_id=sinceId)\n        else:\n            if(not sinceId):\n                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",max_id=str(max_id - 1))\n            else:\n                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",max_id=str(max_id - 1), since_id=sinceId)\n        if not new_tweets:\n            print("No more tweets found")\n            break\n        for tweet in new_tweets:\n            

In [109]:
#https://github.com/agalea91/nhl_2016_playoffs_twitter/blob/master/nhl_playoffs_2016_twitter_study.ipynb
data = {'created_at': [],'text': [], 
        'retweet_count': [], 'tweet_id':[],
        'user_favorites_count': [], 'tweet_location': [],
        'user_followers_count':[], 'user_time_zone':[],
        'user_id':[], 'user_name':[], 'in_reply_to_status_id':[], 
        'in_reply_to_user_id':[], 'in_reply_to_screen_name':[],
        'screen_name':[], 'user_verified':[],
        'user_location':[], 'user_description':[], 
        'user_statuses_count': [], 'user_created_at':[],
        'user_friends_count': [], 'favorite_count': []}

for t in forma:
    data['created_at'].append(t[0])
    data['text'].append(t[1])
    data['retweet_count'].append(t[2])
    data['favorite_count'].append(t[3])
    data['tweet_id'].append(t[4])
    data['user_name'].append(t[5])
    data['screen_name'].append(t[6])
    data['user_description'].append(t[7])
    data['user_id'].append(t[8])
    data['user_location'].append(t[9])
    data['user_favorites_count'].append(t[10])
    data['user_followers_count'].append(t[11])
    data['user_friends_count'].append(t[12])
    data['user_statuses_count'].append(t[13])
    data['user_created_at'].append(t[14])
    data['user_verified'].append(t[15])
    data['user_time_zone'].append(t[16])
    data['tweet_location'].append(t[17])
    data['in_reply_to_status_id'].append(t[18])
    data['in_reply_to_user_id'].append(t[19])
    data['in_reply_to_screen_name'].append(t[20])

df = pd.DataFrame(data)
#df.drop_duplicates(inplace=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df['user_created_at'] = pd.to_datetime(df['user_created_at'])

RT = []
for t in data['text']:
    RT.append(t.split()[0]=='RT')
df['RT'] = RT

df.head()

,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,tweet_location,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,2017-09-18 07:30:05,21,None,None,None,31,THINKgovuk,Home button or no home button 😉 it's still dan...,909680911515602945,None,...,513,22277,448,1322025240,Great Britain,THINK! road safety,2294,Edinburgh,True,False
1,2017-09-21 15:25:56,21,None,None,None,24,HubSpot,iOS 11 is here... and it's 🔥 #ios11 #iphone #i...,910887828636266496,None,...,26325,780793,46113,14458280,"Cambridge, MA",HubSpot,75905,Eastern Time (US & Canada),True,False
2,2017-09-24 11:58:27,2,None,None,None,0,abroudin,Top notch\n#iPhoneX #iPhone #Apple https://t....,911922779213254656,None,...,720,294,481,269516553,"Paris, Ile-de-France",Alexandre Broudin,4176,Paris,False,False
3,2017-09-24 11:54:29,0,None,None,None,1,AmyTayl13397155,RT @eccomiqua1997: iPhone X? Galaxy s8/note8??...,911921778380943360,None,...,1802,23,152,875096143058817024,,Amy Taylor,2025,None,False,True
4,2017-09-24 11:42:18,0,None,None,None,0,sumeet0110,Ever wonder how to clear memory when there is ...,911918715150757889,None,...,55,19,15,76013880,Kuwait,Sumeet Sonawane,8,Mumbai,False,False


In [110]:
df.describe()

,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count
count,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02
mean,0.920000,5.530000,9.116358e+17,3068.540000,11963.590000,2536.320000,1.866137e+17,5.266677e+04
std,3.335393,19.619334,2.786676e+14,8190.147269,78412.649706,6885.353534,3.447521e+17,2.470776e+05
min,0.000000,0.000000,9.096809e+17,0.000000,0.000000,3.000000,6.728130e+05,8.000000e+00
25%,0.000000,0.000000,9.115433e+17,66.750000,136.500000,147.250000,1.282547e+08,5.435000e+02
50%,0.000000,1.000000,9.116580e+17,708.000000,735.500000,469.500000,1.567031e+09,2.210500e+03
75%,0.000000,3.000000,9.118289e+17,2266.000000,3046.250000,2057.500000,3.327674e+09,6.580000e+03
max,21.000000,137.000000,9.119228e+17,67542.000000,780793.000000,47944.000000,9.061879e+17,1.682850e+06


In [111]:
len(np.unique(df.text))

84

In [112]:
df.columns[df.isnull().any()]

Index(['in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'tweet_location', 'user_time_zone'],
      dtype='object')

In [113]:
#https://cloud.google.com/natural-language/docs/sentiment-tutorial

In [114]:
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = True)
df.to_sql('sawi_tweets_historical', engine, if_exists='replace')

2017-09-24 14:16:17,684 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-09-24 14:16:17,689 INFO sqlalchemy.engine.base.Engine {}
2017-09-24 14:16:17,698 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-09-24 14:16:17,703 INFO sqlalchemy.engine.base.Engine {}
2017-09-24 14:16:17,708 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-09-24 14:16:17,712 INFO sqlalchemy.engine.base.Engine {}
2017-09-24 14:16:17,719 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-09-24 14:16:17,723 INFO sqlalchemy.engine.base.Engine {}
2017-09-24 14:16:17,729 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-09-24 14:16:17,733 INFO sqlalchemy.engine.base.Engine {}
2017-09-24 14:16:17,738 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-09-24 14

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


2017-09-24 14:16:18,046 INFO sqlalchemy.engine.base.Engine ({'in_reply_to_user_id': None, 'user_description': 'Official THINK! road safety channel. We encourage safer behaviour to reduce the number of deaths/injuries on our roads. Twitter policy http://t.co/oDLlotPz8f', 'tweet_location': None, 'user_time_zone': 'Edinburgh', 'user_name': 'THINK! road safety', 'user_location': 'Great Britain', 'created_at': datetime.datetime(2017, 9, 18, 7, 30, 5), 'user_verified': 1, 'index': 0, 'user_id': 1322025240, 'user_friends_count': 448, 'in_reply_to_screen_name': None, 'user_favorites_count': 513, 'user_created_at': datetime.datetime(2013, 4, 2, 8, 31, 5), 'screen_name': 'THINKgovuk', 'user_followers_count': 22277, 'RT': 0, 'text': "Home button or no home button 😉 it's still dangerous to use while driving. 📱 #AppleEvent #iPhone #Apple #iPhone8… https://t.co/6PLGEsKzws", 'user_statuses_count': 2294, 'in_reply_to_status_id': None, 'tweet_id': 909680911515602945, 'favorite_count': 21, 'retweet_coun

2017-09-24 14:16:18,153 INFO sqlalchemy.engine.base.Engine COMMIT
